# Comparación entre noticias del mundo y el marca

## Empezamos cargando las noticias de "El Mundo"

In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
DatosNoticias_ElMundo = pd.DataFrame()
DatosNoticias_Marca = pd.DataFrame()

### Las guardamos en la variable 'csv_reader_EM'

In [3]:
with open('.\Data\El Mundo.csv') as csv_El_Mundo:
    csv_reader_EM = csv.reader(csv_El_Mundo, delimiter='|')
    line_count = 0
    for row in csv_reader_EM:
        if line_count == 0:
            DatosNoticias_ElMundo = pd.DataFrame(columns=['Título', 'Autor', 'Contenido', 'Target'], index=range(1,100))
            line_count += 1
        else:
            DatosNoticias_ElMundo['Título'][line_count] = str(row[1])
            DatosNoticias_ElMundo['Autor'][line_count] = str(row[0]).replace(" ", "")
            DatosNoticias_ElMundo['Contenido'][line_count] = str(row[2])
            DatosNoticias_ElMundo['Target'][line_count] = 0
            line_count += 1
    print(f'Processed {line_count} lines.')
line_count_EM = line_count

Processed 100 lines.


In [4]:
DatosNoticias_ElMundo.head(1)

,Título,Autor,Contenido,Target
1,Detenida una pareja tras fingir ser los padres...,EUROPAPRESS,Agentes de la Policía Nacional han detenido a ...,0


## Cargamos también las noticias del periódico deportivo "Marca"

### Las guardamos en la variable 'csv_reader_M'

In [5]:
with open('.\Data\Marca.csv') as csv_Marca:
    csv_reader_M = csv.reader(csv_Marca, delimiter='|')
    line_count = 0
    for row in csv_reader_M:
        if line_count == 0:
            DatosNoticias_Marca = pd.DataFrame(columns=['Título', 'Autor', 'Contenido', 'Target'], index=range(1,100))
            line_count += 1
        else:
            DatosNoticias_Marca['Título'][line_count] = str(row[1])
            DatosNoticias_Marca['Autor'][line_count] = str(row[0]).replace(" ", "")
            DatosNoticias_Marca['Contenido'][line_count] = str(row[2])
            DatosNoticias_Marca['Target'][line_count] = 1
            line_count += 1
    print(f'Processed {line_count} lines.')
line_count_M = line_count

Processed 100 lines.


In [6]:
DatosNoticias_Marca.head(1)

,Título,Autor,Contenido,Target
1,"Álvaro Benito lo tiene claro: ""No ayuda a nadi...",RedacciónMARCA,"Álvaro Benito, comentarista de la Cadena SER y...",1


## Unimos los dos dataframes en un gran dataframe de noticias 

In [7]:
DataFrames = [DatosNoticias_ElMundo, DatosNoticias_Marca]
Noticias = pd.concat(DataFrames)
del(DataFrames,DatosNoticias_ElMundo, DatosNoticias_Marca)

In [8]:
Noticias.head(2)

,Título,Autor,Contenido,Target
1,Detenida una pareja tras fingir ser los padres...,EUROPAPRESS,Agentes de la Policía Nacional han detenido a ...,0
2,Dos policías detenidos en Río como sospechosos...,AFP,Dos policías fueron detenidos el martes en Río...,0


In [9]:
Noticias.tail(2)

,Título,Autor,Contenido,Target
98,Chiellini: I'm 200 percent convinced that Cris...,marca.com,Giorgio Chiellini AFP Juventus captain Giorg...,1
99,"Terapia de Grupo: ""Zidane le echa valor regres...",RadioMARCA,Zidane durante la rueda de prensa de su desped...,1


In [10]:
Noticias['Título'].head()

1    Detenida una pareja tras fingir ser los padres...
2    Dos policías detenidos en Río como sospechosos...
3    Causa el doble de muertes de lo que se creía: ...
4    Lío entre Risto Mejide y la 'antifeminista' So...
5    Eurovisión ya tiene sus favoritos y España no ...
Name: Título, dtype: object

## Convertimos los csv's en un formato que podamos usar en nuestro modelo predictivo (Word Cleaning + HotEnd Encoding)

In [11]:
from sklearn import feature_extraction

#vectorizador_count = feature_extraction.text.CountVectorizer()
#X = vectorizador_count.fit_transform(Noticias['Título'])

vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(Noticias['Título'])
df_titulo_procesado =  pd.DataFrame(texto_vectorizado.toarray(), columns=vectorizador_tfidf.get_feature_names())

vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(Noticias['Autor'])
df_autor_procesado =  pd.DataFrame(texto_vectorizado.toarray(), columns=vectorizador_tfidf.get_feature_names())

vectorizador_tfidf = feature_extraction.text.TfidfVectorizer()
texto_vectorizado = vectorizador_tfidf.fit_transform(Noticias['Contenido'])
df_contenido_procesado =  pd.DataFrame(texto_vectorizado.toarray(), columns=vectorizador_tfidf.get_feature_names())


In [12]:
df_autor_procesado.shape

(198, 102)

In [13]:
df_contenido_procesado.shape

(198, 3248)

In [14]:
Noticias.shape

(198, 4)

In [15]:
Noticias = Noticias.reset_index(drop=True)

In [16]:
Noticias_Procesadas = pd.concat(
[
    Noticias,
    df_titulo_procesado,
    df_autor_procesado,
    df_contenido_procesado
], axis=1, sort=False)

del(df_titulo_procesado,
    df_autor_procesado,
    df_contenido_procesado)

In [17]:
Noticias_Procesadas.shape

(198, 4498)

In [18]:
Noticias_Procesadas.head(2)

,Título,Autor,Contenido,Target,000,10,100,11,12,14,...,ídem,íntima,órdenes,órgano,última,últimas,último,últimos,única,único
0,Detenida una pareja tras fingir ser los padres...,EUROPAPRESS,Agentes de la Policía Nacional han detenido a ...,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Dos policías detenidos en Río como sospechosos...,AFP,Dos policías fueron detenidos el martes en Río...,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
Noticias_num = Noticias_Procesadas
Noticias_num.drop(['Título', 'Autor', 'Contenido'], axis=1).head(5)

,Target,000,10,100,11,12,14,15,18,20,...,ídem,íntima,órdenes,órgano,última,últimas,último,últimos,única,único
0,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0.302373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
X = Noticias_num['Target']
y = Noticias_num.drop(['Target'], axis=1)

In [21]:
X.shape

(198,)

In [22]:
y.shape

(198, 4497)

## Creamos el modelo predictivo usando Keras, y lo entrenamos con nuestros datos

In [26]:
from tensorflow import keras
import keras
# import keras

Using TensorFlow backend.


In [27]:
from keras.models import Sequential
from keras.layers import Dense

In [28]:
modelo = Sequential([
    Dense(units=5, activation='sigmoid', input_dim=4),
    Dense(units=1, activation='sigmoid')
])

In [29]:
modelo.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

In [30]:
historial = modelo.fit(X_std, y , epochs=100)

NameError: name 'X_std' is not defined

In [ ]:
plt.plot(historial.history["acc"])
plt.title("Precisión vs épocas de entrenamiento");

## Comprobamos el porcentaje de acierto de nuestro modelo

## Guardamos nuestro modelo, para poder ser usado posteriormente